In [35]:
# Michelle Kouba
# Weather app - this program asks the user for a zip or city and retrieves local up to the minute weather data.
import requests
from termcolor import colored, cprint

def retrieve_weather(name, latitude, longitude):
    loop = 0
    desired_temp_type = ""
    desired_temp = ""
    while loop < 3:
        loop = loop + 1
        print("")
        print(colored("Would you like to see your weather details in:", 'purple'))
        print("1: Fahrenheit")
        print("2: Celsius")
        print("3: Kelvin")
        print("")
        desired_temp = input("Select your temperature option:")
        if desired_temp == "1":
            desired_temp_type = "imperial"
            break
        elif desired_temp == "2":
            desired_temp_type = "metric"
            break
        elif desired_temp == "3":
            desired_temp_type = "standard"
            break
        else:
            print(" ")
            if loop == 3:
                print(" ")
                print("Invalid entry too many times.")
                break
    # Pulling weather information from the API by latitude and longitude
    if desired_temp == "1" or desired_temp == "2" or desired_temp == "3":
        API_KEY = "3132ff1773f036e2bd46c90a14845676"
        weather_url = f"https://api.openweathermap.org/data/2.5/onecall?lat={latitude}&lon={longitude}&units=" \
                      f"{desired_temp_type}&exclude=hourly,minutely&appid={API_KEY}"

        # Initiating API call
        try:
            get_weather_data = requests.get(weather_url).json()
            response = requests.get(weather_url)
            response.raise_for_status()
        # Call failure
        except requests.exceptions.HTTPError:
            print("The entered request {} does not exist.".format(colored(name, 'red')))
        else:
            # Printing the requested weather conditions
            print("")
            print("-------------------------------------------")
            print(colored(("Current weather condition for " + str(name)), 'green'))
            print("-------------------------------------------")
            print("Current temperature for the day :" + colored(str(get_weather_data['current']['temp']) + str(" degrees"),'green'))
            print("Current pressure :" + colored(str(get_weather_data['current']['pressure']) + str("hPa"), 'green'))
            print("Current humidity :" + colored(str(get_weather_data['current']['humidity']) + str("%"), 'green'))
            print("Current weather description :" + colored(str(get_weather_data['current']['weather'][0]["description"]),'green'))
            print("Forecast Minimum temperature:" + colored(str(get_weather_data['daily'][0]['temp']['min']) + str(" degrees"), 'green'))
            print("Forecast Maximum temperature:" + colored(str(get_weather_data['daily'][0]['temp']['max']) + str(" degrees"), 'green'))
            print("-------------------------------------------")
            print("")

def retrieve_zip_code():
    # Retriev zip code from user
    loop = 0
    while loop < 3:
        loop = loop + 1
        zip_code = input("Enter the zip code:")
        # Checking if entry is valid
        if zip_code.isnumeric():
            if len(zip_code) == 5:
                retrieve_weather_by_zip_code(zip_code)
                break
            else:
                print("You have entered an invalid zip code.")
        else:
            print("You have entered an invalid zip code.")
            if loop == 3:
                print("You've had 3 incorrect attempts.")


def retrieve_weather_by_zip_code(zip_code):
    country_code = "US"
    API_KEY = "3132ff1773f036e2bd46c90a14845676"
    location_url = f"http://api.openweathermap.org/geo/1.0/zip?zip={zip_code},{country_code}&appid={API_KEY}"
    # Checking the API call status
    try:
        zip_code_data = requests.get(location_url).json()
        response = requests.get(location_url)
        response.raise_for_status()
    # API call failure, retrieve error type
    except requests.exceptions.HTTPError:
        print("The entered zip code {} does not exist.".format(colored(zip_code, 'red')))
    else:
        name = zip_code_data['name']
        latitude = zip_code_data['lat']
        longitude = zip_code_data['lon']
        retrieve_weather(name, latitude, longitude)

def retrieve_city_name():
    # Variable that limits the number of cities/states returned for common city names
    limit = "5"
    city_name = input("Enter the city name in US:")
    country_code = "US"
    API_KEY = "3132ff1773f036e2bd46c90a14845676"
    city_name_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name},{country_code}&limit={limit}" \
                    f"&appid={API_KEY}"
    # Checking the API call status
    try:
        city_name_data = requests.get(city_name_url).json()
        response = requests.get(city_name_url)
        response.raise_for_status()
    # API call failure, retrieve error type
    except requests.exceptions.HTTPError:
        print(" ")
        print("The entered zip code {} does not exist.".format(colored(city_name, 'red')))
    else:
        create_list = [f['state'] for f in city_name_data]
        create_list = list(set(create_list))
        create_list.sort()
        # If the city is a common city name, giving the user a list of states to choose from
        if len(create_list) == 1:
            name = city_name_data[0]['name']
            latitude = city_name_data[0]['lat']
            longitude = city_name_data[0]['lon']
            retrieve_weather(name, latitude, longitude)
        elif len(create_list) > 1:
            print("City " + city_name.upper() + " is seen in " + str(len(create_list)) + " states")
            loop_two = 0
            while loop_two < 3:
                for n in range(len(create_list)):
                    text = colored(create_list[n], 'purple')
                    print("If you like to read the weather conditions in " + text + " " + "select :" + colored(
                        (str(n + 1)), 'purple'))
                input_message = int(input("Please select a choice of state:"))
                if 1 <= input_message <= len(create_list):
                    name = city_name_data[input_message - 1]['name']
                    latitude = city_name_data[input_message - 1]['lat']
                    longitude = city_name_data[input_message - 1]['lon']
                    retrieve_weather(name, latitude, longitude)
                    break
                else:
                    if loop_two > 1:
                        print("Too many invalid options.  Goodbye")
                        break
                    else:
                        print("You have selected an invalid option")
                loop_two = loop_two + 1
        else:
            print("{} is not a valid city with weather data".format(colored(city_name, 'red')))

# Main
def main():
    loop = 0
    print("")
    while loop < 3:
        loop = loop + 1
        print(colored("Would you like your weather by:", "purple"))
        print("1: Zip Code")
        print("2: City")
        print("3: Exit")
        print(" ")
        input_option = input("Select your option:")
        print(" ")
        if input_option == "1":
            retrieve_zip_code()
        elif input_option == "2":
            retrieve_city_name()
        elif input_option == "3":
            print("Thank you for visiting.")
            break
        else:
            if loop == 2:
                print("You have entered an invalid option too many times.  Exiting.....")
                break
            else:
                print("You have entered an invalid option: {}".format(colored(input_option, 'red')))
        # Queries user if they want to continue with another city or zip
        loop_item = 0
        while loop_item < 3:
            print(" ")
            input_message = input("Do you like to check the weather for another city (Y/N) : ")
            print(" ")
            if input_message.upper() == "Y":
                loop = 0
                break
            elif input_message.upper() == "N":
                loop = 4
                print("")
                print("------Goodbye------")
                break
            else:
                print("Invalid entry.")
                if loop_item == 3:
                    print("Invalid entry too many times.  Exiting system!")
                    break

# Main
if __name__ == "__main__":
    print("-------Welcome to Weather Express-------")
    main()